In [ ]:
import os
import rasterio as rio
from rasterio.enums import Resampling
import matplotlib.pyplot as plt
import numpy as np
import glob

def rescale_and_save(in_fn):
    fn = in_fn.replace("10cm", "20cm")
    out_fn_rgb = fn.replace(".tif", "_rgb.tif")
    out_fn_nir = fn.replace(".tif", "_nir.tif")
    
    if not os.path.exists(fn.split("/Ortho")[0]):
        os.makedirs(os.path.join(fn.split("/Ortho")[0], "Ortho"))
        
    r =  rio.open(in_fn)
    downscale_factor = 0.5
    r_arr = r.read(
        out_shape=(
                r.count,
                int(r.height * downscale_factor),
                int(r.width * downscale_factor)
            ),
            resampling=Resampling.bilinear
    )


    type(r_arr)
    # r_arr = np.moveaxis(r_arr, 0, -1)

    for i in range(4):
        max = r_arr[i].max()
        r_arr[i] = (r_arr[i] / max) * 255
    # r_arr = np.flip(r_arr[0:3, ...], axis=0)
    R = r_arr[2].copy()
    G = r_arr[1]
    B = r_arr[0].copy()

    r_arr[0] = R
    r_arr[2] = B

    # scale image transform
    transform = r.transform * r.transform.scale(
        (r.width / r_arr.shape[-1]),
        (r.height / r_arr.shape[-2])
    )

    rgb_profile = r.profile.copy()
    rgb_profile.update(
        driver="GTiff",
        height=r_arr.shape[1],
        width=r_arr.shape[2],
        dtype=r_arr.dtype,
        count=3,
        transform=transform
    )

    nir_profile = rgb_profile.copy()
    nir_profile.update(
        count=1
    )

    new_rgb = rio.open(
        out_fn_rgb,
        'w',
        **rgb_profile
    )
    new_nir = rio.open(
        out_fn_nir,
        'w',
        **nir_profile
    )
    new_rgb.write(r_arr[0:3])
    new_nir.write(np.expand_dims(r_arr[3], 0))

In [ ]:
dirs = glob.glob("data/*10cm")
for dir in dirs[1:]:
    fns = glob.glob(os.path.join(dir, "Ortho/*.tif"))
    for fn in fns:
        rescale_and_save(fn)